# Simple Process Tree Builder

This notebook opens a Wintap process parquet file (or files) using duckdb to build a networkx process tree graph.

Workflow is:

1. Map process parquet into a duckdb view
2. Create an iterator for all rows in the set using the wg.add_all() function.
    Note: the current example function is dead simple, but could be greatly enhanced with complex filtering, joining, etc.
3. Pass the iterator to the build graph function which adds nodes with properties and parent->child relationships
4. Display a few simple metics about the graph

## Setup
Download data from https://gdo-wintap.llnl.gov

A good starting set is: https://gdo168.llnl.gov/data/ACME-2023/stdview-20231109-20231111-workshop/

Untar and just use the PROCESS

In [1]:
# Import packages used in notebooks
import duckdb
import networkx as nx
import wintapgraph as wg
%load_ext magic_duckdb

In [2]:
# Initialize an in-memory db. Save reference in a variable and then set magic-duckdb environment. Result is ability to use the same DB instance from python code and %dql/%%dql magics.
con = duckdb.connect()
%dql -co con
# Only uses a process table
%dql create view process as from '~/data/wintapv6/ACME-Redo/stdview-20231109-20231111/process_summary.parquet'
# Display a simple summary of the process_uber_summary table
%dql summarize process


ValueError: Error executing create view process as from '~/data/wintapv6/ACME-2023/stdview-20231109-20231111/process_summary.parquet' in DuckDB

In [ ]:
processes = wg.add_all(con)
netg = wg.build_process_tree_graph(con, processes)
netg

In [ ]:
[len(c) for c in sorted(nx.connected_components(netg), key=len, reverse=True)]

In [ ]:
nx.number_connected_components(netg)

In [ ]:
%dql select count(*) from process